# Plotting RS results
this notebook is mostly for talk figures.

In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb


In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir   = pjoin(os.getcwd(), 'raw_data')


In [ ]:
folder_path = pjoin(output_dir,'All_temporal-UMAP-GAS_plotting+statistics_002')

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))


In [ ]:
fig_dir    = pjoin(folder_path,'figures')
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All figures will be saved in: {}".format(fig_dir))

# Temporal Network

In [ ]:
all_gas = {'cues': [loadPickle('processed_data/RS_null-metrics_GAS-001/temporal_all_gas.pkl'),
                loadPickle('processed_data/RJ_null-metrics_GAS-003/temporal_all_gas.pkl')],
           'MO' :  [loadPickle('processed_data/RS_null-metrics_GAS-recalVelocity-001/temporal_all_gas_MO.pkl'),
                loadPickle('processed_data/RJ_null-metrics_GAS-MO-001/temporal_all_gas_MO.pkl')]}

In [ ]:

all_gas_means = [{'data':{'cues':[],'MO':[]},'null':{'cues':[],'MO':[]}},
                 {'data':{'cues':[],'MO':[]},'null':{'cues':[],'MO':[]}}]
for i in range(2):
    for c in np.arange(5):
        all_gas_means[i]['null']['cues'].append([np.mean([np.mean(y[2]) for y in x[c]]) for x in all_gas['cues'][i]])
        all_gas_means[i]['data']['cues'].append([np.mean([np.mean(y[1]) for y in x[c]]) for x in all_gas['cues'][i]])
        
        all_gas_means[i]['null']['MO'].append([np.mean([np.mean(y[2]) for y in x[c]]) for x in all_gas['MO'][i]])
        all_gas_means[i]['data']['MO'].append([np.mean([np.mean(y[1]) for y in x[c]]) for x in all_gas['MO'][i]])
del all_gas

In [ ]:
savePickle(pjoin(folder_path,'all_gas_means.pkl'),all_gas_means)

In [ ]:
all_embeddings = [loadPickle('processed_data/RS-UMAP-embedding-000/FN_UMAP_embedding.pkl'),
                 loadPickle('processed_data/rj1031209-UMAP-embedding-002/FN_UMAP_embedding.pkl')]

In [ ]:
all_FN_flattened = [loadPickle('processed_data/rockstar_CenterOut_FlattenedFN_wBuffer.pkl'),
                   loadPickle('processed_data/rj1031209_CenterOut_FN-Flattened_10msBin_with500buffer_MIonly-1.5-trialCount.pkl')]

In [ ]:
all_gas_means = loadPickle(pjoin(folder_path,'all_gas_means.pkl'))

Monkey RS: 0 vs 1: [0.21]s
Monkey RS: 0 vs 4: [0.17]s
Monkey RJ: 0 vs 1: [0.27]s
Monkey RJ: 0 vs 4: [0.18]s


In [ ]:
# sns.set_context("paper")
afont = {'fontname':'Roboto','fontsize' : 16,'weight':'bold'}
bfont = {'fontname':'Roboto','fontsize' : 14}
hfont = {'fontname':'Roboto','fontsize' : 20,'weight':'bold'}
cols = 4
rows = 2

subject_names = ['Monkey Rs', 'Monkey Rj']#'Monkey Bx']
fig = plt.figure(figsize=(20,10))


plot_num = 1
gas_bounds = [[0.08,0.20],[0.19,0.31]]
l=3
sig_vals = [[0.21,0.17],[0.27,0.18]] # [0 vs 1,0 vs 4]
sig_colors = []
for row  in range(rows):
    FN_flattened = all_FN_flattened[row]
    data_embedding = all_embeddings[row]
    
    trial_nums = []
    for direction in range(8):
        trials = [x[1] for i,x in enumerate(FN_flattened['FN_info']) if (x[0]==direction)]
        trial_nums.append(max(trials)+1)
    
    for col in range(cols):
        ax = fig.add_subplot(rows,cols,plot_num)

        if col == 0:
            #UMAP
            plt.title(subject_names[row],loc="left",**hfont)
            cmap = sns.color_palette("husl",8)
            start_cmap = sns.husl_palette(8, l=0.1)
            
            start_time = time2bin(0)
            end_time = time2bin(2)

            FN_info = [x for i,x in enumerate(FN_flattened['FN_info']) for d in range(8) for tr in range(trial_nums[d])
                                                                if (x[0] == d
                                                                and  x[1] == tr 
                                                                and x[2] >= start_time
                                                                and x[2] < end_time)]
            for d in range(8):
                for tr in range(10):#range(trial_nums[d]):
                    node_idx = [i for i,x in enumerate(FN_info) if (x[0] == d
                                                                and  x[1] == tr
                                                                and x[2] >= start_time
                                                                and x[2] < end_time)]

                    points = np.array([[x,y] for x,y in zip(data_embedding[node_idx, 0],data_embedding[node_idx, 1])])# points + 0.05*np.random.randn(*points.shape)

                    # Linear length along the line:
                    distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
                    distance = np.insert(distance, 0, 0)/distance[-1]

                    # Build a list of the spline function, one for each dimension:
                    splines = [UnivariateSpline(distance, coords,k=5,s=0.5) for coords in points.T]

                    # Computed the spline for the asked distances:
                    alpha = np.linspace(0, 1, 1000)
                    points_fitted = np.vstack( [spl(alpha) for spl in splines] ).T

                    if row == 0:
                        # plot line using 'scatter' so color can change thorugh time
                        c = sns.blend_palette([start_cmap[d],cmap[d]],as_cmap=True)
                        plt.scatter(points_fitted.T[0],points_fitted.T[1],c=range(len(points_fitted.T[0])),s=1,cmap=c,alpha=0.1)
                        # add arrowhead
                        plt.arrow(points_fitted.T[0,-2], points_fitted.T[1,-2],points_fitted.T[0,-1]-points_fitted.T[0,-2], points_fitted.T[1,-1]-points_fitted.T[1,-2],
                             head_width=0.25,color=cmap[d])
                        
                    if row == 1:
                        # plot line using 'scatter' so color can change thorugh time
                        c = sns.blend_palette([start_cmap[d],cmap[d]],as_cmap=True)
                        plt.scatter(points_fitted.T[1],points_fitted.T[0],c=range(len(points_fitted.T[0])),s=2,cmap=c,alpha=0.1)
                        # add arrowhead
                        plt.arrow(points_fitted.T[1,-2], points_fitted.T[0,-2],points_fitted.T[1,-1]-points_fitted.T[1,-2], points_fitted.T[0,-1]-points_fitted.T[0,-2],
                             head_width=0.3,color=cmap[d])
            plt.axis('equal')
#             if row == 0 :
            plt.xlabel("UMAP X (AU)",**afont)
            plt.ylabel("UMAP Y (AU)",**afont)
            ax.set_yticks([])
            ax.set_xticks([])
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)

            if row == 1 :
                ax.invert_yaxis()
   
        if col == 1:
            # GAS instruction
            start_time = time2bin(-0.1,lastBin=True)
            end_time = time2bin(0.48,lastBin=True)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            legend_labels = ['same','neighboring','opposite']
            
            if row == 0 :
                for i,c in enumerate(dist_vals):
                    x = [bin2time(x,lastBin=True) for x in np.arange(start_time,end_time)]
                    ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l,label=legend_labels[i])
                    ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            else:
                for c in dist_vals:
                    x = [bin2time(x,lastBin=True) for x in np.arange(start_time,end_time)]
                    ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                    ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            
            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(-0.1,0.6,0.1)

            # significance:
            buffer = 0.0025
            plt.plot([sig_vals[row][0],bin2time(end_time,lastBin=True)],\
                     [gas_bounds[row][0]+buffer,gas_bounds[row][0]+buffer], \
                     c=cmap[1], alpha = 0.7, linewidth = 5)
            plt.plot([sig_vals[row][1],bin2time(end_time,lastBin=True)],\
                     [gas_bounds[row][0]+buffer*2,gas_bounds[row][0]+buffer*2], \
                     c=cmap[4],alpha = 0.7, linewidth = 5)
            
            if row == 0:
                leg = ax.legend(frameon=False, handlelength=0,loc='upper left')
                
                for h, t in enumerate(leg.get_texts()):
                    t.set_color(cmap[dist_vals[h]])
                    t.set_weight("bold")
                    t.set_size(16) 

            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Instruction'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            plt.ylabel('Alignment Score',**afont)
            
            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)
            ax.set_yticklabels(labels,**bfont)

            
            plt.ylim(gas_bounds[row])
            sns.despine(ax=ax)
            
        if col == 2:
            # GAS Go

            
            start_time = time2bin(0.9,lastBin=True)
            end_time = time2bin(1.48,lastBin=True)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            
            for c in dist_vals:
                x = [bin2time(x,lastBin=True)-1 for x in np.arange(start_time,end_time)]
                ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)

            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(-0.1,0.6,0.1)
                
            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Go'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)    
            ax.set_yticklabels([])
            plt.ylim(gas_bounds[row])
            sns.despine(ax=ax)
            
        if col == 3:
            # GAS MO
            
            start_time = time2bin(-0.1,lastBin=True,buffer=0.5)
            end_time = time2bin(0.48,lastBin=True,buffer=0.5)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            
            for c in dist_vals:
                x = [bin2time(x,lastBin=True,buffer=0.5) for x in np.arange(start_time,end_time)]
                ax.plot(x,all_gas_means[row]['data']['MO'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                ax.plot(x,all_gas_means[row]['null']['MO'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            
            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)    
            ax.set_yticklabels([])
            ax.set_yticklabels([])
            plt.ylim(gas_bounds[row])
            
            ticks = np.arange(-0.1,0.6,0.1)
            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Move'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            sns.despine(ax=ax)
            
        ax.text(-0.1, 1.1, string.ascii_uppercase[plot_num-1], transform=ax.transAxes, 
        size=20, weight='bold')  
        plot_num += 1

plt.tight_layout()
plt.savefig(pjoin(fig_dir,'temporal_network-letters.png'))

In [ ]:
# sns.set_context("paper")
afont = {'fontname':'Roboto','fontsize' : 16, 'ha':'center','weight':'bold'}
bfont = {'fontname':'Roboto','fontsize' : 14}
hfont = {'fontname':'Roboto','fontsize' : 20,'weight':'bold'}
cols = 3
rows = 2

subject_names = ['Monkey Rs', 'Monkey Rj']#'Monkey Bx']
fig = plt.figure(figsize=(15,10))


plot_num = 1
gas_bounds = [[0.08,0.20],[0.19,0.31]]
l=3
sig_vals = [[0.21,0.17],[0.27,0.18]] # [0 vs 1,0 vs 4]
sig_colors = []

times = [[0,0.5],[0.5,1],[1,2]]
for row  in range(rows):
    FN_flattened = all_FN_flattened[row]
    data_embedding = all_embeddings[row]
    
    trial_nums = []
    for direction in range(8):
        trials = [x[1] for i,x in enumerate(FN_flattened['FN_info']) if (x[0]==direction)]
        trial_nums.append(max(trials)+1)

    #UMAP
    cmap = sns.color_palette("husl",8)
    start_cmap = sns.husl_palette(8, l=0.1)

    start_time = time2bin(0)
    end_time = time2bin(2)

    FN_info = [x for i,x in enumerate(FN_flattened['FN_info']) for d in range(8) for tr in range(trial_nums[d])
                                                        if (x[0] == d
                                                        and  x[1] == tr 
                                                        and x[2] >= start_time
                                                        and x[2] < end_time)]
    for col,ti in enumerate(times):
        ax = fig.add_subplot(rows,cols,plot_num)

        if row==0:
            ax.set_title(f"+{ti[0]} s to +{ti[1]} s",**afont)
        for d in range(8):
            for tr in range(10):#range(trial_nums[d]):
                node_idx = [i for i,x in enumerate(FN_info) if (x[0] == d
                                                            and  x[1] == tr
                                                            and x[2] >= start_time
                                                            and x[2] < end_time)]

                points = np.array([[x,y] for x,y in zip(data_embedding[node_idx, 0],data_embedding[node_idx, 1])])# points + 0.05*np.random.randn(*points.shape)

                # Linear length along the line:
                distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
                distance = np.insert(distance, 0, 0)/distance[-1]

                # Build a list of the spline function, one for each dimension:
                splines = [UnivariateSpline(distance, coords,k=5,s=0.5) for coords in points.T]

                # Computed the spline for the asked distances:
                alpha = np.linspace(0, 1, len(node_idx)*5)
                points_fitted = np.vstack( [spl(alpha) for spl in splines] ).T

                if row == 1:
                    temp = np.copy(points_fitted)
                    points_fitted.T[0] = temp.T[1]
                    points_fitted.T[1] = temp.T[0]
                
                #gray/past time
                plot_start_time = 0
                plot_end_time = time2bin(ti[0],buffer=0)*5
                
                points2plot_x = points_fitted.T[0][plot_start_time:plot_end_time]
                points2plot_y = points_fitted.T[1][plot_start_time:plot_end_time]
                c = sns.blend_palette([start_cmap[d],cmap[d]],as_cmap=True)
                plt.scatter(points2plot_x,points2plot_y,c=range(len(points2plot_x)),s=1,cmap='Greys',alpha=0.05)
                
                #### colored arrow
                plot_start_time = time2bin(ti[0],buffer=0)*5
                plot_end_time = time2bin(ti[1],buffer=0)*5
                
                points2plot_x = points_fitted.T[0][plot_start_time:plot_end_time]
                points2plot_y = points_fitted.T[1][plot_start_time:plot_end_time]
                
                c = sns.blend_palette([start_cmap[d],cmap[d]],as_cmap=True)
                plt.scatter(points2plot_x,points2plot_y,c=range(len(points2plot_x)),s=1,cmap=c,alpha=1)
                # add arrowhead
                plt.arrow(points2plot_x[-2], points2plot_y[-2],\
                          points2plot_x[-1]-points2plot_x[-2],\
                          points2plot_y[-1]-points2plot_y[-2],\
                            head_width=0.25,color=cmap[d])

                plt.axis('equal')
                
                if row == 1 and col == 0:
                    plt.xlabel("UMAP X (AU)",**afont)
#                     plt.ylabel("UMAP Y (AU)",**afont)
#                     ax.yaxis.set_label_coords(0.1,0.5, transform=ax.transAxes)
                    ax.text(0, 0.5, "UMAP Y (AU)", transform=ax.transAxes, rotation='vertical', size=16, weight='bold')
                
                ax.set_yticks([])
                ax.set_xticks([])
                ax.spines['right'].set_visible(False)
                ax.spines['top'].set_visible(False)
                ax.spines['bottom'].set_visible(False)
                ax.spines['left'].set_visible(False)
        
        if col == 0:
            ax.text(-0.1, 0.5, subject_names[row], transform=ax.transAxes, size=20, weight='bold', rotation='vertical')
        if row == 1 :
            ax.invert_yaxis()
        if row == 0 :
            ax.text(-0.1, 1.1, string.ascii_uppercase[plot_num-1], transform=ax.transAxes, size=20, weight='bold')
        ax.axis('equal')
        plot_num += 1
plt.tight_layout()
plt.savefig(pjoin(fig_dir,'temporal-network-embedding.png'))

In [ ]:
# sns.set_context("paper")
afont = {'fontname':'Roboto','fontsize' : 16,'weight':'bold'}
bfont = {'fontname':'Roboto','fontsize' : 14}
hfont = {'fontname':'Roboto','fontsize' : 20,'weight':'bold'}
cols = 3
rows = 2

subject_names = ['Monkey Rs', 'Monkey Rj']#'Monkey Bx']
fig = plt.figure(figsize=(15,10))


plot_num = 1
gas_bounds = [[0.08,0.20],[0.19,0.31]]
l=3
sig_vals = [[0.21,0.17],[0.27,0.18]] # [0 vs 1,0 vs 4]
sig_colors = []
for row  in range(rows):
    FN_flattened = all_FN_flattened[row]
    # data_embedding = all_embeddings[row]
    
    trial_nums = []
    for direction in range(8):
        trials = [x[1] for i,x in enumerate(FN_flattened['FN_info']) if (x[0]==direction)]
        trial_nums.append(max(trials)+1)
    
    for col in range(cols):
        ax = fig.add_subplot(rows,cols,plot_num)

        if col == 0:
            # GAS instruction
            start_time = time2bin(-0.1,lastBin=True)
            end_time = time2bin(0.48,lastBin=True)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            legend_labels = ['same','neighboring','opposite']
            
            if row == 0 :
                for i,c in enumerate(dist_vals):
                    x = [bin2time(x,lastBin=True) for x in np.arange(start_time,end_time)]
                    ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l,label=legend_labels[i])
                    ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            else:
                for c in dist_vals:
                    x = [bin2time(x,lastBin=True) for x in np.arange(start_time,end_time)]
                    ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                    ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            
            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(-0.1,0.6,0.1)

            # significance:
            buffer = 0.0025
            plt.plot([sig_vals[row][0],bin2time(end_time,lastBin=True)],\
                     [gas_bounds[row][0]+buffer,gas_bounds[row][0]+buffer], \
                     c=cmap[1], alpha = 0.7, linewidth = 5)
            plt.plot([sig_vals[row][1],bin2time(end_time,lastBin=True)],\
                     [gas_bounds[row][0]+buffer*2,gas_bounds[row][0]+buffer*2], \
                     c=cmap[4],alpha = 0.7, linewidth = 5)
            
            if row == 0:
                leg = ax.legend(frameon=False, handlelength=0,loc='upper left')
                
                for h, t in enumerate(leg.get_texts()):
                    t.set_color(cmap[dist_vals[h]])
                    t.set_weight("bold")
                    t.set_size(16) 

            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Instruction'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            # plt.ylabel('Alignment Score',**afont)
            ax.set_ylabel(f'{subject_names[row]}\nAlignment Score', multialignment='center',**afont)
            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)
            ax.set_yticklabels(labels,**bfont)

            
            plt.ylim(gas_bounds[row])
            sns.despine(ax=ax)
            
        if col == 1:
            # GAS Go

            
            start_time = time2bin(0.9,lastBin=True)
            end_time = time2bin(1.48,lastBin=True)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            
            for c in dist_vals:
                x = [bin2time(x,lastBin=True)-1 for x in np.arange(start_time,end_time)]
                ax.plot(x,all_gas_means[row]['data']['cues'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                ax.plot(x,all_gas_means[row]['null']['cues'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)

            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(-0.1,0.6,0.1)
                
            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Go'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)    
            ax.set_yticklabels([])
            plt.ylim(gas_bounds[row])
            sns.despine(ax=ax)
            
        if col == 2:
            # GAS MO
            
            start_time = time2bin(-0.1,lastBin=True,buffer=0.5)
            end_time = time2bin(0.48,lastBin=True,buffer=0.5)
            
            dist_vals = [0,1,4]
            cmap = sns.color_palette("viridis",5)
            
            for c in dist_vals:
                x = [bin2time(x,lastBin=True,buffer=0.5) for x in np.arange(start_time,end_time)]
                ax.plot(x,all_gas_means[row]['data']['MO'][c][start_time:end_time],'-',c=cmap[c],linewidth=l)
                ax.plot(x,all_gas_means[row]['null']['MO'][c][start_time:end_time],'--',c=cmap[c],linewidth=l)
            
            ax.plot([0,0],gas_bounds[row],'k', linewidth=5, alpha=0.2)
            ticks = np.arange(gas_bounds[row][0],gas_bounds[row][1]+0.02,0.02)
            labels = [round(x,2)  for x in ticks]
            ax.set_yticks(ticks)    
            ax.set_yticklabels([])
            ax.set_yticklabels([])
            plt.ylim(gas_bounds[row])
            
            ticks = np.arange(-0.1,0.6,0.1)
            labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
            labels[int(np.where(ticks==0)[0])] = 'Move'

            ax.set_xticks(ticks)
            ax.set_xticklabels(labels,**afont)
            ax.get_xticklabels()[1].set_weight("bold")

            sns.despine(ax=ax)
        if row==0:    
            ax.text(-0.1, 1.1, string.ascii_uppercase[plot_num-1], transform=ax.transAxes, 
            size=20, weight='bold')  
        plot_num += 1

plt.tight_layout()
plt.savefig(pjoin(fig_dir,'temporal_network-GAS-letters-subject.png'))

In [ ]:
subject_names = ['Monkey RS', 'Monkey RJ']

for s in range(len(all_gas['cues'])):
    dist_vals = [1,4]
    
    for d in dist_vals:
        dist_0 = [[y[1] for y in x[0]] for x in all_gas['cues'][s]]
        c = d
        dist_1 = [[y[1] for y in x[c]] for x in all_gas['cues'][s]]
#         null_gas = [[np.mean(y[2]) for y in x[c]] for x in gas_cues]
        
        p = [stats.mannwhitneyu(a,b)[1] for a,b in zip(dist_0,dist_1)]
        
        p_corr = multipletests(p,0.001,method='bonferroni')
        
        x = [bin2time(x,lastBin=True) for x in np.arange(len(p))]
        
#         plt.figure()
#         plt.plot(x,p_corr[0])
        
#         print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(x)[np.argwhere(p_corr[0]==True)])}s')
#         print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(x)[np.argwhere(p_corr[0]==True)])}s')
        print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(p_corr[1])[np.argwhere(p_corr[0]==False)])}')

In [ ]:
# max p-val that is significant ( p<=_____)
subject_names = ['Monkey RS', 'Monkey RJ']

for s in range(len(all_gas['cues'])):
    dist_vals = [1,4]
    
    for d in dist_vals:
        dist_0 = [[y[1] for y in x[0]] for x in all_gas['cues'][s]]
        c = d
        dist_1 = [[y[1] for y in x[c]] for x in all_gas['cues'][s]]
#         null_gas = [[np.mean(y[2]) for y in x[c]] for x in gas_cues]
        
        p = [stats.mannwhitneyu(a,b)[1] for a,b in zip(dist_0,dist_1)]
        
        p_corr = multipletests(p,0.001,method='bonferroni')
        
        x = [bin2time(x,lastBin=True) for x in np.arange(len(p))]
        
#         plt.figure()
#         plt.plot(x,p_corr[0])
        
#         print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(x)[np.argwhere(p_corr[0]==True)])}s')
#         print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(x)[np.argwhere(p_corr[0]==True)])}s')
        print(f'{subject_names[s]}: 0 vs {d}: {max(np.array(p_corr[1])[np.argwhere(p_corr[0]==True)])}')

In [ ]:
# time of significance
subject_names = ['Monkey RS', 'Monkey RJ']
for s in range(len(all_gas_cues)):
    dist_vals = [1,4]
    
    for d in dist_vals:
        dist_0 = [[y[1] for y in x[0]] for x in all_gas_cues[s]]
        c = d
        dist_1 = [[y[1] for y in x[c]] for x in all_gas_cues[s]]
#         null_gas = [[np.mean(y[2]) for y in x[c]] for x in gas_cues]
        
        p = [stats.mannwhitneyu(a,b)[1] for a,b in zip(dist_0,dist_1)]

        p_corr = multipletests(p,0.001,method='bonferroni')

        x = [bin2time(x,lastBin=True) for x in np.arange(len(p))]
        
        plt.figure()
        plt.plot(x,p_corr[0])
        
        print(f'{subject_names[s]}: 0 vs {d}: {min(np.array(x)[np.argwhere(p_corr[0]==True)])}s')